In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()


MachineLearningAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



In [2]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [3]:
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [4]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")

ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)

fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()

    index  threshold     score  confidence    class
0     116  -0.086805 -0.135812        1.56  outlier
1     117  -0.086805 -0.075292        0.87   normal
2     118  -0.086805 -0.028934        0.33   normal
3     119  -0.086805 -0.109206        1.26  outlier
4     120  -0.086805 -0.042102        0.49   normal
5     121  -0.086805 -0.053488        0.62   normal
6     122  -0.086805 -0.020840        0.24   normal
7     123  -0.086805 -0.060657        0.70   normal
8     124  -0.086805 -0.059104        0.68   normal
9     125  -0.086805 -0.019291        0.22   normal
10    126  -0.086805 -0.051708        0.60   normal
11    127  -0.086805 -0.067164        0.77   normal
12    128  -0.086805 -0.035623        0.41   normal
13    129  -0.086805 -0.006108        0.07   normal
14    130  -0.086805 -0.020975        0.24   normal
15    131  -0.086805 -0.052229        0.60   normal
16    132  -0.086805 -0.032066        0.37   normal


Addition of new training data based on previous prediction results

In [5]:
#ana.add_prediction()
#print(ana)

In [6]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [7]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
ana.predict(test2_data, test2_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

   index  threshold     score  confidence    class
0    266  -0.086805 -0.096806        1.12  outlier
1     47  -0.086805 -0.106128        1.22  outlier
2     17  -0.086805 -0.145112        1.67  outlier
3    119  -0.086805 -0.109206        1.26  outlier
4     16  -0.086805 -0.130434        1.50  outlier


In [8]:
#ana.add_prediction()
#print(ana)

In [9]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()

In [1]:
from src.StreamPort.machine_learning.methods import MachineLearningEvaluateModelStabilityNative

%store -r routine_summary #error_summary

model_eval = MachineLearningEvaluateModelStabilityNative(test_records=routine_summary)
true_classes, stability_score = model_eval.run()

print("\nModel stability score: ", stability_score)

true_classes["num_tests"] = routine_summary.groupby("index").cumcount() + 1

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


no stored variable or alias #error_summary
Model performance Summary:     index  threshold     score  confidence    class  \
0     14  -0.083773 -0.113353        1.35  outlier   
1     48  -0.090027 -0.056211        0.62   normal   
2     48  -0.083773 -0.044790        0.53   normal   
3     50  -0.090027  0.025127       -0.28   normal   
4     50  -0.090027  0.025127       -0.28   normal   

                         date class_true  stability_score  
0  2025-07-24 10-17-44-644731    not set         0.545455  
1  2025-07-24 10-09-18-367740    not set         0.950176  
2  2025-07-24 10-17-51-506789                    0.950176  
3  2025-07-24 10-08-57-566591    not set         1.000000  
4  2025-07-24 10-09-18-367740                    1.000000  

Model stability score:  0.6962619266053752


In [2]:
import plotly.graph_objects as go

fig = go.Figure()

for i in range(len(true_classes)):
    current_index = true_classes.index[i]
    this_group_in_summary = routine_summary[routine_summary["index"] == current_index]
    print("this Index: ", current_index, " and group: ", this_group_in_summary)
    if this_group_in_summary.empty:
        continue

    # X axis is repetition count: 1, 2, ..., N
    x_vals = list(range(1, len(this_group_in_summary) + 1))

    # Map 'class' to 0/1
    y1 = this_group_in_summary["class"].map({"normal": 0, "outlier": 1}).tolist()
    y2 = this_group_in_summary["confidence"].tolist()

    # Class trace (left Y axis)
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y1,
        mode="lines+markers",
        yaxis="y1",
        marker=dict(symbol="circle", color="red"),
        line=dict(dash="dot"),
        hovertext=this_group_in_summary["class"],
        hoverinfo="text"
    ))

    # Confidence trace (right Y axis)
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y2,
        mode="lines+markers",
        yaxis="y2",
        marker=dict(symbol="square", color="blue"),
        line=dict(dash="solid"),
        hovertemplate="Confidence: %{y}<extra></extra>"
    ))

# Layout setup
fig.update_layout(
    title="True Class (0/1) and Confidence by Test Repetition",
    xaxis=dict(title="Test Repetition Number"),
    yaxis=dict(
        title="Class (0=Normal, 1=Outlier)",
        tickvals=[0, 1],
        ticktext=["Normal", "Outlier"],
        tickfont=dict(color="red")
    ),
    yaxis2=dict(
        title="Confidence Score",
        overlaying="y",
        side="right",
        range=[0.4, 1.6],
        tickfont=dict(color="blue")
    ),
    template="simple_white",
    height=500,
    width=1000,
    legend=dict(
        x=0.5,
        y=1.1,
        xanchor="center",
        yanchor="top",
        orientation="h"
    ),
    showlegend = False
)

fig.show()


this Index:  14  and group:      index  threshold     score  confidence    class  \
40     14  -0.083773 -0.113353        1.35  outlier   

                          date  
40  2025-07-24 10-17-44-644731  
this Index:  48  and group:      index  threshold     score  confidence   class                        date
22     48  -0.090027 -0.056211        0.62  normal  2025-07-24 10-09-18-367740
47     48  -0.083773 -0.044790        0.53  normal  2025-07-24 10-17-51-506789
this Index:  50  and group:      index  threshold     score  confidence   class                        date
1      50  -0.090027  0.025127       -0.28  normal  2025-07-24 10-08-57-566591
20     50  -0.090027  0.025127       -0.28  normal  2025-07-24 10-09-18-367740
this Index:  118  and group:      index  threshold     score  confidence   class                        date
16    118  -0.090027 -0.060224        0.67  normal  2025-07-24 10-09-11-893602
this Index:  119  and group:      index  threshold     score  confidence  

In [ ]:
# import shap
# from src.StreamPort.machine_learning.methods import MachineLearningExplainModelShap

# shap_explainer = MachineLearningExplainModelShap(model = "classification", model_type = "tree")
# ana = shap_explainer.run(ana)
# shap_values = ana.data["shap_values"]
# print(shap_values)
#shap.summary_plot(shap_values, test2_data)